In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

In [2]:
from src.transformers import modeling_with_arc_xlnet

In [3]:
#from src.transformers import modeling_xlnet

In [4]:
import argparse
import glob
import logging
import os
import random

In [5]:
#!pip install spacy

In [6]:
from kg_loader import KG
from graph_utils import GraphEncoder

In [7]:
from src.transformers.tokenization_xlnet import XLNetTokenizer
from src.transformers.tokenization_bert import BertTokenizer
from src.transformers.tokenization_roberta import RobertaTokenizer

In [8]:
from src.transformers.modeling_with_arc_xlnet import XLNetForMultipleChoice
from src.transformers.modeling_roberta import RobertaForMultipleChoice
from src.transformers.modeling_bert import BertForMultipleChoice

In [9]:
from src.transformers.optimization import AdamW, get_linear_schedule_with_warmup

In [10]:
from src.transformers.configuration_xlnet import XLNetConfig
from src.transformers.configuration_bert import BertConfig
from src.transformers.configuration_roberta import RobertaConfig

In [11]:
from src.transformers.file_utils import WEIGHTS_NAME

In [12]:
from utils_arc import convert_examples_to_features, processors

In [13]:
from src.transformers.tokenization_utils import PreTrainedTokenizer

In [14]:
try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

In [15]:
logger = logging.getLogger(__name__)

In [16]:
ALL_MODELS = sum(
    (
        tuple(conf.pretrained_config_archive_map.keys())
        for conf in (BertConfig, RobertaConfig, XLNetConfig)
    ),
    (),
)

In [17]:
MODEL_CLASSES = {
    "bert" : (BertConfig, BertForMultipleChoice, BertTokenizer),
    "XLNet" : (XLNetConfig, XLNetForMultipleChoice, XLNetTokenizer),
    "Roberta" : (RobertaConfig, RobertaForMultipleChoice, RobertaTokenizer),
}

In [18]:
def select_field(features, field):
    return [
        [
            choice[field] 
            for choice in feature.choices_features
        ]
        for feature in features
]

In [19]:
preds = np.array([1,4,2,3,0,3,2,1,2,3,3,2,4,1,2,1,3,2,2,2,1,1,3,3,2,2,1,3,3,2,2,3,1,3]).astype('int64')
labels = np.array([3,2,4,1,1,3,1,2,2,4,1,2,4,1,4,1,4,1,2,3,4,1,4,1,4,1,2,3,2,3,1,2,3,1]).astype('int64')

In [20]:
def simple_accuracy(preds, labels):
    print("labels {} \n".format(labels))
    print("label.shape {} \n".format(labels.shape))
    print("label data type {} \n".format(labels.dtype))
    print("preds shape {} \n".format(preds.shape))
    print("preds data type {} \n".format(preds.dtype))
    print(preds)
    return (preds == labels).mean()

In [21]:
x = simple_accuracy(preds,labels)

labels [3 2 4 1 1 3 1 2 2 4 1 2 4 1 4 1 4 1 2 3 4 1 4 1 4 1 2 3 2 3 1 2 3 1] 

label.shape (34,) 

label data type int64 

preds shape (34,) 

preds data type int64 

[1 4 2 3 0 3 2 1 2 3 3 2 4 1 2 1 3 2 2 2 1 1 3 3 2 2 1 3 3 2 2 3 1 3]


In [22]:
x

0.2647058823529412

In [23]:
#def simple_accuracy(out, labels):
    #outputs = np.argmax(out, axis=0)
    #return np.sum(outputs == labels)

In [24]:
seed = 42
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)

In [25]:
def train(train_dataset, model, tokenizer):
    # Train the model
    if local_rank in [-1,0]:
        tb_writer = SummaryWriter()

    train_batch_size = per_gpu_train_batch_size * max(1, n_gpu)
    train_sampler = RandomSampler(train_dataset) if local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler = train_sampler, batch_size = train_batch_size)
    
    num_train_epochs = 3.0
    if max_steps > 0:
        t_total = max_steps
        num_train_epochs = max_steps // (len(train_dataloader) // gradient_accumulation_steps) + 1

    else:
        t_total = len(train_dataloader) // gradient_accumulation_steps * num_train_epochs

    # Prepare optimizer and schedule ( Linear warmpu and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate, eps = adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps = t_total
    )

    if fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[local_rank], output_device=local_rank, find_unused_parameters=True
        )

    # Train
    logger.info("******* Running Training *********")
    logger.info(" Num Examples = %d", len(train_dataset))
    logger.info(" Num Epochs = %d", num_train_epochs)
    logger.info(" Instantaneous batch size per GPU = %d", per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        train_batch_size
        * gradient_accumulation_steps
        * (torch.distributed.get_world_size() if local_rank != -1 else 1),
    )

    logger.info(" Gradient Accumulation steps = %d", gradient_accumulation_steps)
    logger.info(" Total optimization steps =%d",t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    best_dev_acc = 0.0
    best_steps = 0
    model.zero_grad()
    train_iterator = trange(int(num_train_epochs), desc ="Epoch", 
                            disable=local_rank not in [-1, 0])
    set_seed(seed) # Added here for reproductibility
    
    ent = torch.from_numpy(np.stack([kg.kg_embeddings['ent_embeddings'] 
                                      for _ in range(1)],axis=0)).to(device)
    rel = torch.from_numpy(np.stack([kg.kg_embeddings['rel_matrices'] 
                                      for _ in range(1)],axis=0)).to(device)
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", 
                              disable=local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(device) for t in batch)
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2]
                if model_type in ["bert", "xlnet"]
                else None,  # XLM don't use segment_ids
                
                "gpre" : batch[3],
                "ghyp" : batch[4],
                "ent" : ent,
                "rel" : rel,
                "labels": batch[5],
            }

            outputs = model(**inputs)
            loss = outputs[0] # model outputs are always tuple in transformers

            if n_gpu > 1:
                loss = loss.mean() # mean() to average on multi-gpu parallel training
            if gradient_accumulation_steps > 1:
                loss = loss / gradient_accumulation_steps

            if fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), max_grad_norm)
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

            tr_loss += loss.item()
            if(step +1) % gradient_accumulation_steps == 0:
                optimizer.step()
                scheduler.step() # update learning rate scheduler
                model.zero_grad()
                global_step += 1

                if local_rank in [-1,0] and logging_steps > 0 and global_step % logging_steps == 0:
                    # Log metrics
                    # Log metrics
                    if (
                        local_rank == -1 and evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                        if results["eval_acc"] > best_dev_acc:
                            best_dev_acc = results["eval_acc"]
                            best_steps = global_step
                            if do_test:
                                results_test = evaluate(model, tokenizer, test=True)
                                for key, value in results_test.items():
                                    tb_writer.add_scalar("test_{}".format(key), value, global_step)
                                logger.info(
                                    "test acc: %s, loss: %s, global steps: %s",
                                    str(results_test["eval_acc"]),
                                    str(results_test["eval_loss"]),
                                    str(global_step),
                                )
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / logging_steps, global_step)
                    logger.info(
                        "Average loss: %s at global step: %s",
                        str((tr_loss - logging_loss) / logging_steps),
                        str(global_step),
                    )
                    logging_loss = tr_loss

                if local_rank in [-1, 0] and save_steps > 0 and global_step % save_steps == 0:
                    # Save model checkpoint
                    print(output_dir)
                    output_dir1 = os.path.join(output_dir, "checkpoint-{}".format(global_step))
                    print(output_dir1)
                    if not os.path.exists(output_dir1):
                        os.makedirs(output_dir1)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir1)
                    tokenizer.save_vocabulary(output_dir1)
                    torch.save(output_dir,os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir1)
            if max_steps > 0 and global_step > max_steps:
                epoch_iterator.close()
                break
    if local_rank in [-1,0]:
        tb_writer.close()

    return global_step, tr_loss / global_step, best_steps

In [26]:
def evaluate(model, tokenizer, prefix="", test = False):
    eval_task_names = (task_name,)
    eval_outputs_dirs = (output_dir,)

    results = {}
    for eval_task, eval_output_dir in zip(eval_task_names, eval_outputs_dirs):
        eval_dataset = load_and_cache_examples(
            eval_task, tokenizer, evaluate=True, test=False)

        if not os.path.exists(eval_output_dir) and local_rank in [-1, 0]:
            os.makedirs(eval_output_dir)

        eval_batch_size = per_gpu_eval_batch_size * max(1, n_gpu)
        print("eval_batch_size {} \n".format(eval_batch_size))
        # Note that DistributedSampler samples randomly
        eval_sampler = SequentialSampler(eval_dataset)
        eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=eval_batch_size)

        # multi-gpu evaluate
        if n_gpu > 1:
            model = torch.nn.DataParallel(model)

        # Eval!
        logger.info("***** Running evaluation {} *****".format(prefix))
        logger.info("  Num examples = %d", len(eval_dataset))
        logger.info("  Batch size = %d", eval_batch_size)
        eval_loss = 0.0
        nb_eval_steps = 0
        preds = None
        out_label_ids = None
        ent = torch.from_numpy(np.stack([kg.kg_embeddings['ent_embeddings'] 
                                          for _ in range(1)],axis=0)).to(device)
        rel = torch.from_numpy(np.stack([kg.kg_embeddings['rel_matrices'] 
                                          for _ in range(1)],axis=0)).to(device)
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            model.eval()
            batch = tuple(t.to(device) for t in batch)

            with torch.no_grad():
                inputs = {
                    "input_ids": batch[0],
                    "attention_mask": batch[1],
                    "token_type_ids": batch[2]
                    if model_type in ["bert", "xlnet"]
                    else None,  # XLM don't use segment_ids
                    "gpre" : batch[3],
                    "ghyp" : batch[4],
                    "ent" : ent,
                    "rel" : rel,
                    "labels": batch[5],
                }
                outputs = model(**inputs)
                tmp_eval_loss, logits = outputs[:2]

                eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1
            if preds is None:
                preds = logits.detach().cpu().numpy()
                out_label_ids = inputs["labels"].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

        eval_loss = eval_loss / nb_eval_steps
        preds = np.argmax(preds, axis=1)
        acc = simple_accuracy(preds, out_label_ids)
        result = {"eval_acc": acc, "eval_loss": eval_loss}
        results.update(result)

        output_eval_file = os.path.join(eval_output_dir, "is_test_" + str(test).lower() + "_eval_results.txt")

        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results {} *****".format(str(prefix) + " is test:" + str(test)))
            writer.write("model           =%s\n" % str(model_name_or_path))
            writer.write(
                "total batch size=%d\n"
                % (
                    per_gpu_train_batch_size
                    * gradient_accumulation_steps
                    * (torch.distributed.get_world_size() if local_rank != -1 else 1)
                )
            )
            writer.write("train num epochs=%d\n" % num_train_epochs)
            writer.write("fp16            =%s\n" % fp16)
            writer.write("max seq length  =%d\n" % max_seq_length)
            for key in sorted(result.keys()):
                logger.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))
    return results


In [27]:
def load_and_cache_examples(task, tokenizer, evaluate=False, test=False):
    if local_rank not in [-1, 0]:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    processor = processors[task]()
    # Load data features from cache or dataset file
    if evaluate:
        cached_mode = "dev"
    elif test:
        cached_mode = "test"
    else:
        cached_mode = "train"
    assert not (evaluate and test)
    cached_features_file = os.path.join(
        data_dir,
        "cached_{}_{}_{}_{}".format(
            cached_mode,
            list(filter(None, model_name_or_path.split("/"))).pop(),
            str(max_seq_length),
            str(task),
        ),
    )
    if os.path.exists(cached_features_file) and not overwrite_cache:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", data_dir)
        label_list = processor.get_labels()
        if evaluate:
            examples = processor.get_dev_examples(data_dir)
        elif test:
            examples = processor.get_test_examples(data_dir)
        else:
            examples = processor.get_train_examples(data_dir)
        logger.info("Training number: %s", str(len(examples)))
        features = convert_examples_to_features(
                    examples,
                    label_list,
                    max_seq_length,
                    tokenizer,
                    graph_encoder,
                    kg,
                    pad_on_left=bool(model_type in ["xlnet"]),
                    pad_token_segment_id=4 if model_type in ["xlnet"] else 0,
                    
                )
        #print(features[0].choices_features)
        if local_rank in [-1, 0]:
            logger.info("Saving features into cached file %s", cached_features_file)
            torch.save(features, cached_features_file)

    if local_rank == 0:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor(select_field(features, "input_ids"), dtype=torch.long)
    print("all_input_ids.shape {} \n".format(all_input_ids.shape))
    all_input_mask = torch.tensor(select_field(features, "attention_mask"), dtype=torch.long)
    print("all_input_mask.shape {} \n".format(all_input_mask.shape))
    all_segment_ids = torch.tensor(select_field(features, "token_type_ids"), dtype =torch.long)
    print("all_segment_ids.shape {} \n".format(all_segment_ids.shape))
    all_prem_ids = torch.tensor(select_field(features,"gpre"), dtype = torch.long)
    print("all_prem_ids.shape {} \n".format(all_prem_ids.shape))
    all_hyp_ids = torch.tensor(select_field(features,"ghyp"),dtype = torch.long)
    print("all_hyp_ids.shape {} \n".format(all_hyp_ids.shape))
    all_label_ids = torch.tensor([f.label for f in features], dtype=torch.long)
    print("all_label_ids.shape {} \n".format(all_label_ids.shape))
    # to do changes here:
    dataset = TensorDataset(all_input_ids,all_input_mask, all_segment_ids, all_prem_ids, all_hyp_ids, all_label_ids,)
        
    return dataset

In [28]:
task_name = 'arc'

In [29]:
data_dir = "data/arc/processed"

In [30]:
model_type = "XLNet"

In [31]:
MODEL_CLASSES.keys()

dict_keys(['bert', 'XLNet', 'Roberta'])

In [32]:
model_name_or_path = 'xlnet-base-cased'
do_lower_case = False # set true when using uncased model

In [33]:
#ALL_MODELS

In [34]:
processors.keys()

dict_keys(['arc', 'race', 'swag'])

In [35]:
task_name = 'arc'

In [36]:
output_dir = "save/"

In [37]:
config_name = ""

In [38]:
do_train = True
do_eval = True
do_test = True

In [39]:
evaluate_during_training = False

In [40]:
per_gpu_train_batch_size = 1

In [41]:
per_gpu_eval_batch_size = 1

In [42]:
gradient_accumulation_steps = 1

In [43]:
learning_rate = 5e-5

In [44]:
weight_decay = 0.0

In [45]:
adam_epsilon = 1e-8

In [46]:
max_grad_norm = 1.0

In [47]:
num_train_epochs = 3.0

In [48]:
#If > 0: set total number of training steps to perform. Override num_train_epochs.
max_steps = -1

In [49]:
warmup_steps = 0

In [50]:
logging_steps = 50

In [51]:
save_steps = 1000

In [52]:
eval_all_checkpoints = True

In [53]:
no_cuda = False

In [54]:
cache_dir = ""

In [55]:
overwrite_output_dir = True

In [56]:
overwrite_cache = True

In [57]:
fp16 = False

In [58]:
fp16_opt_level = False

In [59]:
path_to_kg = "data/conceptnet"

In [60]:
max_ent_pre = 262

In [61]:
max_ent_hyp = 83

In [62]:
local_rank = -1

In [63]:
server_ip = ""
server_port = ""

In [64]:
if (
        os.path.exists(output_dir)
        and os.listdir(output_dir)
        and do_train
        and not overwrite_output_dir
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                output_dir
            )
        )

In [65]:
if server_ip and server_port:
        # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
        import ptvsd

        print("Waiting for debugger attach")
        ptvsd.enable_attach(address=(server_ip, server_port), redirect_output=True)
        ptvsd.wait_for_attach()

In [66]:
if local_rank == -1 or no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
        n_gpu = torch.cuda.device_count()
    # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
else:
    torch.cuda.set_device(local_rank)
    device = torch.device("cuda", local_rank)
    torch.distributed.init_process_group(backend="nccl")
    n_gpu = 1
device = device

In [67]:
device

device(type='cuda')

In [68]:
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if local_rank in [-1, 0] else logging.WARN,
    )
logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    local_rank,
    device,
    n_gpu,
    bool(local_rank != -1),
    fp16,
)

01/07/2020 14:18:03 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 4, distributed training: False, 16-bits training: False


In [69]:
set_seed(seed)

In [70]:
task_name = task_name.lower()

In [71]:
if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))
processor = processors[task_name]()
label_list = processor.get_labels()
num_labels = len(label_list)

In [72]:
label_list

['0', '1', '2', '3']

In [73]:
num_labels

4

In [74]:
if local_rank not in [-1, 0]:
    torch.distributed.barrier() # Make sure only the first process in distributed training will download model and vocab

In [75]:
model_type = model_type.lower()
config_class, model_class, tokenizer_class = MODEL_CLASSES['XLNet']

In [76]:
config = config_class.from_pretrained(
    config_name if config_name else model_name_or_path,
    num_labels=num_labels,
    finetuning_task=task_name,
    cache_dir=cache_dir if cache_dir else None,
)

01/07/2020 14:18:04 - INFO - filelock -   Lock 139974592477968 acquired on /home/abhishek/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.ef1824921bc0786e97dc88d55eb17aabf18aac90f24bd34c0650529e7ba27d6f.lock
01/07/2020 14:18:04 - INFO - filelock -   Lock 139974592477968 released on /home/abhishek/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.ef1824921bc0786e97dc88d55eb17aabf18aac90f24bd34c0650529e7ba27d6f.lock
01/07/2020 14:18:04 - INFO - src.transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json from cache at /home/abhishek/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.ef1824921bc0786e97dc88d55eb17aabf18aac90f24bd34c0650529e7ba27d6f
01/07/2020 14:18:04 - INFO - src.transformers.configuration_utils -   Model config {
  "attn_type": "bi",
  "bi_data": false,
  "bos_to

In [77]:
tokenizer_name=""

In [78]:
max_ent_pre = 262
max_ent_hyp = 83

In [79]:
tokenizer = tokenizer_class.from_pretrained(
    tokenizer_name if tokenizer_name else model_name_or_path,
    do_lower_case=do_lower_case,
    cache_dir=cache_dir if cache_dir else None,

)

01/07/2020 14:18:05 - INFO - filelock -   Lock 139974592503056 acquired on /home/abhishek/.cache/torch/transformers/dad589d582573df0293448af5109cb6981ca77239ed314e15ca63b7b8a318ddd.8b10bd978b5d01c21303cc761fc9ecd464419b3bf921864a355ba807cfbfafa8.lock
01/07/2020 14:18:05 - INFO - filelock -   Lock 139974592503056 released on /home/abhishek/.cache/torch/transformers/dad589d582573df0293448af5109cb6981ca77239ed314e15ca63b7b8a318ddd.8b10bd978b5d01c21303cc761fc9ecd464419b3bf921864a355ba807cfbfafa8.lock
01/07/2020 14:18:05 - INFO - src.transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-spiece.model from cache at /home/abhishek/.cache/torch/transformers/dad589d582573df0293448af5109cb6981ca77239ed314e15ca63b7b8a318ddd.8b10bd978b5d01c21303cc761fc9ecd464419b3bf921864a355ba807cfbfafa8


In [80]:
kg = KG(path_to_kg)
graph_encoder = GraphEncoder(kg)

In [81]:
model = model_class.from_pretrained(
    model_name_or_path,
    kg = kg,
    from_tf=bool(".ckpt" in model_name_or_path),
    config = config,
    cache_dir=cache_dir if cache_dir else None,
    )

01/07/2020 14:18:08 - INFO - filelock -   Lock 139973964894352 acquired on /home/abhishek/.cache/torch/transformers/24197ba0ce5dbfe23924431610704c88e2c0371afa49149360e4c823219ab474.7eac4fe898a021204e63c88c00ea68c60443c57f94b4bc3c02adbde6465745ac.lock
01/07/2020 14:18:08 - INFO - filelock -   Lock 139973964894352 released on /home/abhishek/.cache/torch/transformers/24197ba0ce5dbfe23924431610704c88e2c0371afa49149360e4c823219ab474.7eac4fe898a021204e63c88c00ea68c60443c57f94b4bc3c02adbde6465745ac.lock
01/07/2020 14:18:08 - INFO - src.transformers.modeling_utils -   loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-pytorch_model.bin from cache at /home/abhishek/.cache/torch/transformers/24197ba0ce5dbfe23924431610704c88e2c0371afa49149360e4c823219ab474.7eac4fe898a021204e63c88c00ea68c60443c57f94b4bc3c02adbde6465745ac


num_entities 1100498
num_relations 38


01/07/2020 14:18:17 - INFO - src.transformers.modeling_utils -   Weights of XLNetForMultipleChoice not initialized from pretrained model: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias', 'gmatch.ent_emb.weight', 'gmatch.linear1.weight', 'gmatch.linear1.bias', 'EntAttn.linear.weight', 'EntAttn.linear.bias', 'RelAttn.linear.weight', 'RelAttn.linear.bias', 'RelAttn.linear2.weight', 'RelAttn.linear2.bias']
01/07/2020 14:18:17 - INFO - src.transformers.modeling_utils -   Weights from pretrained model not used in XLNetForMultipleChoice: ['lm_loss.weight', 'lm_loss.bias']


In [82]:
model_class

src.transformers.modeling_with_arc_xlnet.XLNetForMultipleChoice

In [83]:
model.to(device)

XLNetForMultipleChoice(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, ele

In [84]:
logger.info("Training/evaluation parameters %s")
best_step = 0

01/07/2020 14:18:20 - INFO - __main__ -   Training/evaluation parameters %s


In [85]:
max_seq_length = 512

In [86]:
if do_train:
    train_dataset = load_and_cache_examples(task_name,tokenizer, evaluate =False)
    global_step, tr_loss, best_steps = train(train_dataset, model, tokenizer)
    logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

01/07/2020 14:18:20 - INFO - __main__ -   Creating features from dataset file at data/arc/processed
01/07/2020 14:18:20 - INFO - utils_arc -   LOOKING AT data/arc/processed train
01/07/2020 14:18:20 - INFO - __main__ -   Training number: 3353
01/07/2020 14:18:56 - INFO - __main__ -   Saving features into cached file data/arc/processed/cached_train_xlnet-base-cased_512_arc


all_input_ids.shape torch.Size([3353, 4, 512]) 

all_input_mask.shape torch.Size([3353, 4, 512]) 

all_segment_ids.shape torch.Size([3353, 4, 512]) 

all_prem_ids.shape torch.Size([3353, 4, 262]) 

all_hyp_ids.shape torch.Size([3353, 4, 83]) 

all_label_ids.shape torch.Size([3353]) 



01/07/2020 14:19:06 - INFO - __main__ -   ******* Running Training *********
01/07/2020 14:19:06 - INFO - __main__ -    Num Examples = 3353
01/07/2020 14:19:06 - INFO - __main__ -    Num Epochs = 3
01/07/2020 14:19:06 - INFO - __main__ -    Instantaneous batch size per GPU = 1
01/07/2020 14:19:06 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 4
01/07/2020 14:19:06 - INFO - __main__ -    Gradient Accumulation steps = 1
01/07/2020 14:19:06 - INFO - __main__ -    Total optimization steps =2517
Iteration:   0%|          | 0/839 [00:00<?, ?it/s]/home/abhishek/anaconda3/envs/QA/lib/python3.7/site-packages/torch/nn/functional.py:1280: UserWarning: `eps` parameter is deprecated and has no effect.
  warnings.warn("`eps` parameter is deprecated and has no effect.")
/home/abhishek/anaconda3/envs/QA/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will 

Iteration:  18%|█▊        | 149/839 [01:52<08:14,  1.39it/s]01/07/2020 14:21:00 - INFO - __main__ -   Average loss: 1.452704312801361 at global step: 150

Iteration:  24%|██▎       | 199/839 [02:28<07:39,  1.39it/s]01/07/2020 14:21:36 - INFO - __main__ -   Average loss: 1.5057471764087678 at global step: 200

Iteration:  28%|██▊       | 233/839 [02:53<07:19,  1.38it/s]


Iteration:  30%|██▉       | 249/839 [03:04<07:04,  1.39it/s]01/07/2020 14:22:12 - INFO - __main__ -   Average loss: 1.404422128200531 at global step: 250

Iteration:  36%|███▌      | 299/839 [03:41<06:30,  1.38it/s]01/07/2020 14:22:48 - INFO - __main__ -   Average loss: 1.45770738363266 at global step: 300

Iteration:  42%|████▏     | 349/839 [04:17<05:53,  1.39it/s]01/07/2020 14:23:25 - INFO - __main__ -   Average loss: 1.4880651128292084 at global step: 350

Iteration:  42%|████▏     | 356/839 [04:22<05:46,  1.39it/s]


...

01/07/2020 14:43:27 - INFO - src.transformers.modeling_utils -   Model weights saved in save/checkpoint-2000/pytorch_model.bin
01/07/2020 14:43:27 - INFO - __main__ -   Saving model checkpoint to save/checkpoint-2000

Iteration:  44%|████▍     | 371/839 [04:32<05:41,  1.37it/s]01/07/2020 14:44:03 - INFO - __main__ -   Average loss: 1.4193358933925628 at global step: 2050

Iteration:  50%|█████     | 421/839 [05:08<05:04,  1.37it/s]01/07/2020 14:44:39 - INFO - __main__ -   Average loss: 1.3685736060142517 at global step: 2100

Iteration:  53%|█████▎    | 442/839 [05:23<04:47,  1.38it/s]


Iteration:  56%|█████▌    | 471/839 [05:44<04:27,  1.37it/s]01/07/2020 14:45:16 - INFO - __main__ -   Average loss: 1.3918996739387512 at global step: 2150

Iteration:  62%|██████▏   | 521/839 [06:21<03:52,  1.37it/s]01/07/2020 14:45:52 - INFO - __main__ -   Average loss: 1.4313869047164918 at global step: 2200

Iteration:  68%|██████▊   | 567/839 [06:54<03:18,  1.37it/s]


Iteration:  68%|██████▊   | 571/839 [06:57<03:14,  1.38it/s]01/07/2020 14:46:29 - INFO - __main__ -   Average loss: 1.4209796690940857 at global step: 2250

Iteration:  74%|███████▍  | 621/839 [07:33<02:36,  1.39it/s]01/07/2020 14:47:05 - INFO - __main__ -   Average loss: 1.3734148907661439 at global step: 2300

Iteration:  80%|███████▉  | 671/839 [08:10<02:00,  1.39it/s]01/07/2020 14:47:41 - INFO - __main__ -   Average loss: 1.4272548580169677 at global step: 2350

Iteration:  82%|████████▏ | 690/839 [08:23<01:47,  1.39it/s]


Iteration:  86%|████████▌ | 721/839 [08:46<01:25,  1.38it/s]01/07/2020 14:48:17 - INFO - __main__ -   Average loss: 1.3906275737285614 at global step: 2400

Iteration:  92%|█████████▏| 771/839 [09:22<00:49,  1.37it/s]01/07/2020 14:48:54 - INFO - __main__ -   Average loss: 1.4145712208747865 at global step: 2450

Iteration:  97%|█████████▋| 815/839 [09:54<00:18,  1.31it/s]


Iteration:  98%|█████████▊| 821/839 [09:59<00:13,  1.37it/s]01/07/2020 14:49:30 - INFO - __main__ -   Average loss: 1.4241965866088868 at global step: 2500

Epoch: 100%|██████████| 3/3 [30:36<00:00, 612.12s/it]
01/07/2020 14:49:43 - INFO - __main__ -    global_step = 2517, average loss = 1.435550414644445


In [88]:
results = {}
if do_eval and local_rank in [-1, 0]:
    if not do_train:
        output_dir = model_name_or_path
    checkpoints = [output_dir]
    if eval_all_checkpoints:
        checkpoints = list(
            os.path.dirname(c) for c in sorted(glob.glob(output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
        )
        logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
    logger.info("Evaluate the following checkpoints: %s", checkpoints)
    for checkpoint in checkpoints:
        global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
        prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

        model = model_class.from_pretrained(checkpoint)
        model.to(device)
        result = evaluate(args, model, tokenizer, prefix=prefix)
        result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
        results.update(result)

01/07/2020 14:50:53 - INFO - __main__ -   Evaluate the following checkpoints: ['save/checkpoint-1000', 'save/checkpoint-2000']
01/07/2020 14:50:53 - INFO - src.transformers.configuration_utils -   loading configuration file save/checkpoint-1000/config.json
01/07/2020 14:50:53 - INFO - src.transformers.configuration_utils -   Model config {
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 0,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "do_sample": false,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_ids": 0,
  "ff_activation": "gelu",
  "finetuning_task": "arc",
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "mem_len": null,
  "n_head": 12,
  "n_layer": 12,
  "num_beams": 1,
  "num_labels": 4,
  "num_return_sequences": 1,
  "output_attentions": false,
 

AttributeError: 'NoneType' object has no attribute 'kg_embeddings'

In [89]:
checkpoints = [output_dir]

In [91]:
checkpoints = list(
            os.path.dirname(c) for c in sorted(glob.glob(output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
        )

In [92]:
checkpoints

['save/checkpoint-1000', 'save/checkpoint-2000']

In [93]:
checkpoint = checkpoints[0]

In [95]:
global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

In [97]:
model = model_class.from_pretrained(checkpoint)

01/07/2020 15:26:34 - INFO - src.transformers.configuration_utils -   loading configuration file save/checkpoint-1000/config.json
01/07/2020 15:26:34 - INFO - src.transformers.configuration_utils -   Model config {
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 0,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "do_sample": false,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_ids": 0,
  "ff_activation": "gelu",
  "finetuning_task": "arc",
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "mem_len": null,
  "n_head": 12,
  "n_layer": 12,
  "num_beams": 1,
  "num_labels": 4,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "pruned_heads": {},
  "repetition_penalty": 1.0,


AttributeError: 'NoneType' object has no attribute 'kg_embeddings'

In [98]:
model_class

src.transformers.modeling_with_arc_xlnet.XLNetForMultipleChoice